In [1]:
MODEL_FILE = 'USELModel_201908071556.h5'
MODEL_DIR = '../data'
EMBEDDING_DIM = 512

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import tensorflow_hub as hub
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Layer
import tensorflow.keras.layers as layers
from tensorflow.keras.optimizers import Adam, Adagrad

In [17]:
class USEEmbeddingLayer(Layer):
    def __init__(self, trainable=True, **kwargs):
        self.dimensions = EMBEDDING_DIM
        super(USEEmbeddingLayer, self).__init__(**kwargs)
        self.trainable=trainable

    def build(self, input_shape):
        self.use = hub.Module('https://tfhub.dev/google/universal-sentence-encoder-large/3', trainable=True,
                               name="{}_module".format(self.name))
        wts = tf.trainable_variables(scope=".*{}_module/.*".format(self.name))
        wts = [v for v in wts if not ("SNLI" in v.name or "SHARED_RANK_ANSWER" in v.name)]
        if len(wts)>0:
          if self.trainable:
            self._trainable_weights += wts
          else:
            self._non_trainable_weights += wts
        else:
          print('WARNING: No weights in Embedding Layer')
        super(USEEmbeddingLayer, self).build(input_shape)

   
    def call(self, x, mask=None):
        result = self.use(tf.squeeze(tf.cast(x, tf.string), axis=1))
        return result

    def compute_mask(self, inputs, mask=None):
        return tf.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [48]:
def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = USEEmbeddingLayer(trainable=False)(input_text)
  dense1 = layers.Dense(512, activation='relu')(embedding)

  dropout0 = layers.Dropout(0)(dense1)
  pred = layers.Dense(1, activation='sigmoid')(dropout0)

  model = Model(inputs=[input_text], outputs=pred)

  return model

In [49]:
model = build_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1001 13:23:58.309897 140256719836992 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [50]:
init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run([init, table_init])
K.set_session(sess)

model_path = os.path.join(MODEL_DIR, MODEL_FILE)
model.load_weights(model_path)

In [51]:
def sex(s):
    it = model.predict(np.array([s]).reshape(-1,1))[0][0]
    return(it)

In [53]:
tweet = 'omg omg omg omg i cried so much, my awkward, estranged, my adorable chocolate uterus. '+\
    'everyone loves closed eye chiffon skirts. omg omg i hate them so '+\
    'yet i feel so alone. omg omg herbals with fashioinable pistils'
1-sex(tweet)

0.9581755362451077

In [54]:
tweet='false logic fucked football optimize deep essential solid golf '+\
    'unproven can entirely fucked simulate with deadweight Becker Friedman loaded AK-47 or fucked ass'
sex(tweet)

0.9628115